In [1]:
import pandas as pd


In [7]:
path = '../data/option3_data/train.csv'
#df_ntrain = pd.read_csv(path,dtype={'Context':str, 'Ground Truth Utterance':str, 'Distractor_0':str, 'Distractor_1':str,
#       'Distractor_2':str, 'Distractor_3':str, 'Distractor_4':str, 'Distractor_5':str,
#       'Distractor_6':str, 'Distractor_7':str, 'Distractor_8':str})
df_train = pd.read_csv(path)

In [8]:
df.head()

,Context,Ground Truth Utterance,Distractor_0,Distractor_1,Distractor_2,Distractor_3,Distractor_4,Distractor_5,Distractor_6,Distractor_7,Distractor_8
0,The thing is my orher hamstring is fine and fl...,"if it is an injury, you'll still need to equal...",So my mouth isn't gonna fall off? Hah,&gt;I'm also going to keep drinking the lemon ...,"Not particularly high intensity, I did some pu...","i guess so.. But apart from anxiety, i got acc...",I think mild concussion is defined by a consta...,[NICE CKS info](http://cks.nice.org.uk/dyspeps...,"You are very much not alone. Personally, I occ...",I've seen many neurologists about my headaches...,Could it at least get me something for the pai...
1,&gt; amitriptyline This medication has an unco...,Good catch. I had it before amitriptyline. Alt...,"Yeah, it's not happening unless a verified doc...",#####&amp;#009; ######&amp;#009; ####&amp;#009...,I would recommend going in and seeing someone ...,"Sounds like you had some trouble in school, wh...",That looks like a spreading infection possibly...,No hair in those two tiny spots. No skin probl...,Unless she's really gross or you have a rare m...,"Good for you, Brother.",go to a different hospital. be persistent. i k...
2,Ok i feel like an idiot writting this and im p...,I have no idea whether or not you guts will ho...,No prob :),I don't have anymore and my health insurance d...,Me too... I'm not sure which is the better opt...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC11...,Why do you think this is a brainstem glioma? I...,I think its an infected lymph node. Its soften...,You're going to be fine. There are only about ...,Depends if you become an alcoholic from it.,"Like you mentioned, it sounds like a panic att..."
3,Age 20 Sex Male Height 172 Cm Weight 60 Kg Rac...,OP: Some personal anecdotes regarding the stom...,How long have you had these symptoms? What was...,you still interested in this?,1. Absolutely not 2. Learn to workout correctl...,the cells in the testicles have some degree of...,If you are having those symptoms contact a doc...,Hmm. It's hard to tell what he came into conta...,My biggest fear is that the 3+ years of heavy ...,So I can make my own. Would you know of anything?,"It's called ptosis. In itself, its not dangero..."
4,The first time I had one there was tenderness ...,"Okay, you have me feeling scared now. I have j...",How long have you been clean? Are you active i...,"I'm not a doctor, but those symptoms sound a l...","Not a practice area specialty, just based on s...",https://plus.google.com/u/0/photos/11327507635...,Seeing your physiatrist for your medication or...,This. Thread closed.,These don't match the symptoms of colon cancer...,I'm on my last year of postgraduate surgical e...,NOT A DOCTOR But I did watch a movie named Lor...


In [25]:
for c in df.columns:
    df[c] = df.apply(lambda r: str(r))

In [19]:
df.columns = ['Context', 'Ground Truth Utterance', 'Distractor_0', 'Distractor_1',
       'Distractor_2', 'Distractor_3', 'Distractor_4', 'Distractor_5',
       'Distractor_6', 'Distractor_7', 'Distractor_8']

In [20]:
df.to_csv(path, index=False)